In [1]:
import requests

auth_url = "https://signin.tradestation.com/authorize"

public_key = "r4bJ08Nbz9f8b6djDhoyCmazNnrrLFL4"
private_key = "hFBk8xgV_UGJEUFnxVW-AFz6YToqZwdvM-48x5wLUQhzKiR99r2w780hL0giBfvd"

params = {"response_type": "code", "client_id": public_key, "redirect_uri": "http://localhost", "audience": "https://api.tradestation.com", "state": "STATE", 
        "scope": "openid offline_access profile MarketData ReadAccount Trade Crypto Matrix OptionSpreads"}

response = requests.get(auth_url, params=params)
response.history
for resp in response.history:
        print(resp.status_code, resp.url)

302 https://signin.tradestation.com/authorize?response_type=code&client_id=r4bJ08Nbz9f8b6djDhoyCmazNnrrLFL4&redirect_uri=http%3A%2F%2Flocalhost&audience=https%3A%2F%2Fapi.tradestation.com&state=STATE&scope=openid+offline_access+profile+MarketData+ReadAccount+Trade+Crypto+Matrix+OptionSpreads


In [ ]:
import requests

token_url = "https://signin.tradestation.com/oauth/token"

authorization_code = "xxMZvFaqtX95pCIj"

headers = {"content-type": "application/x-www-form-urlencoded"}
data = {"grant_type": "authorization_code", "client_id": public_key, "client_secret": private_key, "code": authorization_code, "redirect_uri": "http://localhost"}

r = requests.post(token_url, headers=headers, data=data).json()
print(r)

access_token = r["access_token"]
refresh_token = r["refresh_token"]
id_token = r["id_token"]

In [2]:
import requests
refresh_token = "lzijZJe0VUnjFEzq6dqWK-Q3rqhzyoq7kkrDkqVoJvoYn"
def refresh():
    token_url = "https://signin.tradestation.com/oauth/token"

    headers = {"content-type": "application/x-www-form-urlencoded"}
    data = {"grant_type": "refresh_token", "client_id": public_key, "client_secret": private_key, "refresh_token": refresh_token}

    r = requests.post(token_url, headers=headers, data=data).json()

    access_token = r["access_token"]
    return access_token

In [ ]:
import requests
import pandas as pd
import json
import csv

url = "https://api.tradestation.com/v3/marketdata/barcharts/TSLA"

headers = {"Authorization": f"Bearer {access_token}"}

params = {"unit": "Minute", "firstdate": "2016-12-05T21:00:00Z", "lastdate": "2016-12-06T21:00:00Z"}

response = requests.request("GET", url, headers=headers, stream=True, params=params)
print(response)
if response.status_code == 401:
    access_token = refresh()
    headers = {"Authorization": f"Bearer {access_token}"}
    response = requests.request("GET", url, headers=headers, stream=True, params=params)
    print(response)
response = response.json()
    
fieldnames = list(response["Bars"][0].keys())
with open("tradestation_data/tsla_2016_12_05_2016_12_06.csv", "w", newline="") as csvfile:
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)

    writer.writeheader()
    for row in response["Bars"]:
        writer.writerow(row)

In [ ]:
def stream_lvl1_quotes(symbol):
    url = f"https://api.tradestation.com/v3/marketdata/stream/quotes/{symbol}"
    while True:
        try:
            access_token = refresh()
            headers = {"Authorization": f"Bearer {access_token}"}

            response = requests.get(url, headers=headers, stream=True)
            if response.status_code != 200:
                print(f"{symbol} {response} {response.text}")
                continue   

            for line in response.iter_lines():
                if line:
                    row = json.loads(line)
                    if "Error" in row:
                        print(f"{symbol} {response} {response.text}")
                        continue
                    if "Heartbeat" in row:
                        continue
                    print(row)
        except Exception as e:
            print(f"{symbol} {e}")
            continue

In [ ]:
import os
import csv
import json
import pandas as pd
import datetime
import time

def stream_lvl2_quotes(symbol):
    url = f"https://api.tradestation.com/v3/marketdata/stream/marketdepth/aggregates/{symbol}"
    while True:
        try:
            access_token = refresh()
            headers = {"Authorization": f"Bearer {access_token}"}

            response = requests.get(url, headers=headers, stream=True)
            if response.status_code != 200:
                print(f"{symbol} {response} {response.text}")
                continue            
            
            for line in response.iter_lines():
                if line:
                    bids_and_asks = json.loads(line)
                    if "Heartbeat" in bids_and_asks:
                        continue
                    print(bids_and_asks)
        except Exception as e:
            print(f"{symbol} {e}")
            break
                    
symbols = "TSLA"
stream_lvl2_quotes(symbols)

In [5]:
import requests
account_id = "11655345"
def buy_share(symbol):
    access_token = refresh()

    url = "https://api.tradestation.com/v3/orderexecution/orders"

    payload = {
        "AccountID": "11655345",
        "Symbol": f"{symbol}",
        "Quantity": "1",
        "OrderType": "Market",
        "TradeAction": "BUY",
        "TimeInForce": {"Duration": "DAY"},
        "Route": "Intelligent"
    }
    headers = {
        "content-type": "application/json",
        "Authorization": f"Bearer {access_token}"
    }

    response = requests.request("POST", url, json=payload, headers=headers)

    print(response.text)
buy_share("TSLA")

{"Orders":[{"Message":"Sent order: Buy 1 TSLA @ Market","OrderID":"967793406"}]}



In [11]:
import requests
def sell_share(symbol):
    access_token = refresh()

    url = "https://api.tradestation.com/v3/orderexecution/orders"

    payload = {
        "AccountID": "11655345",
        "Symbol": f"{symbol}",
        "Quantity": "1",
        "OrderType": "Market",
        "TradeAction": "SELL",
        "TimeInForce": {"Duration": "DAY"},
        "Route": "Intelligent"
    }
    headers = {
        "content-type": "application/json",
        "Authorization": f"Bearer {access_token}"
    }

    response = requests.request("POST", url, json=payload, headers=headers)

    print(response.text)
sell_share("TSLA")

{"Orders":[{"Message":"Sent order: Sell 1 TSLA @ Market","OrderID":"967802392"}]}



In [13]:
import requests
def check_status():
    access_token = refresh()
    url = "https://api.tradestation.com/v3/brokerage/accounts/11655345/orders"

    headers = {"Authorization": f"Bearer {access_token}"}

    response = requests.request("GET", url, headers=headers)

    print(response.text)
check_status()

{"Orders":[{"AccountID":"11655345","CommissionFee":"0","ClosedDateTime":"2023-06-07T19:53:10Z","Currency":"USD","Duration":"DAY","FilledPrice":"0","Legs":[{"QuantityOrdered":"1","ExecQuantity":"0","QuantityRemaining":"0","BuyOrSell":"Buy","Symbol":"TSLA","AssetType":"STOCK"}],"OrderID":"967805697","OpenedDateTime":"2023-06-07T19:53:10Z","OrderType":"Market","PriceUsedForBuyingPower":"224.98","RejectReason":"EC501: Day trading margin rules, this trade can possibly designate you as a pattern day trader, please call the Trade Desk at 800-871-3563 for more information.","Routing":"Intelligent","Status":"REJ","StatusDescription":"Rejected","UnbundledRouteFee":"0"},{"AccountID":"11655345","CommissionFee":"0","ClosedDateTime":"2023-06-07T19:53:00Z","Currency":"USD","Duration":"DAY","FilledPrice":"224.9699","Legs":[{"OpenOrClose":"Open","QuantityOrdered":"1","ExecQuantity":"1","QuantityRemaining":"0","BuyOrSell":"Buy","Symbol":"TSLA","AssetType":"STOCK","ExecutionPrice":"224.9699"}],"OrderID":

In [44]:
import datetime
import requests
account_id = "11655345"
def store_price(symbol):
    url = "https://api.tradestation.com/v3/orderexecution/orderconfirm"
    access_token = refresh()
    payload = {
        "AccountID": account_id,
        "Symbol": symbol,
        "Quantity": "1",
        "OrderType": "Market",
        "TradeAction": "BUY",
        "TimeInForce": {"Duration": "DAY"},
        "Route": "Intelligent"
    }
    headers = {"content-type": "application/json", "Authorization": f"Bearer {access_token}"}

    order_time = datetime.datetime.utcnow()
    time = order_time.replace(second=order_time.second - order_time.second%10).strftime("%Y-%m-%dT%H:%M:%SZ")
    response = requests.request("POST", url, json=payload, headers=headers).json()
    print(response)
    print(response["Confirmations"][0])
    buy_price = response["Confirmations"][0]["EstimatedPrice"]
    print(buy_price)
store_price("TSLA")

{'Confirmations': [{'OrderAssetCategory': 'EQUITY', 'Currency': 'USD', 'Route': 'Intelligent', 'TimeInForce': {'Duration': 'DAY'}, 'AccountID': '11655345', 'OrderConfirmID': 'WJ4EfcDjhUOE5ta3RebvZQ', 'EstimatedPrice': '226.53', 'EstimatedCost': '226.53', 'DebitCreditEstimatedCost': '226.53', 'EstimatedCommission': '0', 'SummaryMessage': 'Buy 1 TSLA @ Market'}]}
{'OrderAssetCategory': 'EQUITY', 'Currency': 'USD', 'Route': 'Intelligent', 'TimeInForce': {'Duration': 'DAY'}, 'AccountID': '11655345', 'OrderConfirmID': 'WJ4EfcDjhUOE5ta3RebvZQ', 'EstimatedPrice': '226.53', 'EstimatedCost': '226.53', 'DebitCreditEstimatedCost': '226.53', 'EstimatedCommission': '0', 'SummaryMessage': 'Buy 1 TSLA @ Market'}
226.53


In [ ]:
import requests

access_token = refresh()
account_id = "11655345"
url = f"https://api.tradestation.com/v3/brokerage/stream/accounts/{account_id}/orders"

headers = {"Authorization": f"Bearer {access_token}"}

response = requests.request("GET", url, headers=headers, stream=True)

for line in response.iter_lines():
    if line:
        print(line)

In [2]:
from test_mongo import MongoExplorer
mongo = MongoExplorer()

In [51]:
import numpy as np
import pandas as pd
def eval_signal(symbol, dates, hour, price_dif_thresholds, size_threshold):
    collection_1 = f"{symbol}_10sec_ts_lvl1"
    collection_2 = f"{symbol}_10sec_ts_lvl2"
    df_dict = {}
    df_len = 0
    for date in dates:
        start = f'{date}T{hour}:00:00Z'
        end = f'{date}T{hour+1}:05:00Z'
        query_1 = {'$and': [{'CurTime': {'$gte': start}}, {'CurTime': {'$lt': end}}]}
        df_1 = pd.DataFrame(list(mongo.mongoDB[collection_1].find(query_1)))
        cols = ["Time", "Last", "TotalSize", "Dif"]
        for k in range(1, 25):
            df_1[f"{k}0sec Dif"] = df_1["Last"].shift(-k) - df_1["Last"]
            df_1[f"{k}0sec Sign"] = df_1[f"{k}0sec Dif"] >= 0
            cols.append(f"{k}0sec Dif")
            cols.append(f"{k}0sec Sign")

        end = f'{date}T{hour+1}:00:00Z'
        query_2 = {'$and': [{'CurTime': {'$gte': start}}, {'CurTime': {'$lt': end}}]}
        df_2 = pd.DataFrame(list(mongo.mongoDB[collection_2].find(query_2)))
        df = pd.merge(df_1, df_2, on="CurTime")
        df = df.rename(columns={"CurTime": "Time"})
        df.loc[df["Side"] == "Bid", "Dif"] = (df["Price"] - df["Last"])
        df.loc[df["Side"] == "Ask", "Dif"] = (df["Last"] - df["Price"])
        df.loc[df["Side"] == "Ask", "TotalSize"] *= -1
        df = df[cols]
        df_dict[date] = df

    arr = np.zeros((300, 49))
    arr[:, 0] = price_dif_thresholds
    for i, price_dif_threshold in enumerate(price_dif_thresholds):
        temp_arr = np.zeros((0, 25))
        sign_count = np.zeros(24)
        total_count = 0
        for j, date in enumerate(dates):
            df = df_dict[date].copy()
            df = df.loc[df["Dif"] >= price_dif_threshold]
            if len(df) == 0:
                continue
            df["VolumeSum"] = df["TotalSize"] * df["Dif"]
            cols = ["Time", "Last", "VolumeSum"]
            cols_dict = {"VolumeSum": "sum", "Last": "first"}
            for k in range(1,25):
                cols.append(f"{k}0sec Dif")
                cols.append(f"{k}0sec Sign")
                cols_dict[f"{k}0sec Dif"] = "first"
                cols_dict[f"{k}0sec Sign"] = "first"
            df = df[cols]
            df = df.groupby("Time").agg(cols_dict).reset_index()
            df["Sign"] = df["VolumeSum"] >=0
            cols = ["VolumeSum"]
            for k in range(1,25):
                df["Equal"] = df["Sign"] == df[f"{k}0sec Sign"]
                sign_count[k-1] += df["Equal"].values.sum()
                cols.append(f"{k}0sec Dif")
            total_count += len(df)
            temp_arr = np.concatenate((temp_arr, df[cols].to_numpy()))
        if total_count == 0:
            continue
        temp_df = pd.DataFrame(temp_arr)
        corr_df = temp_df.corr()
        arr[i, 1:25] = corr_df.to_numpy()[0, 1:]
        arr[i, 25:49] = sign_count / total_count

    cols = ["Threshold"]
    for k in range(1,25):
        cols.append(f"{k}0sec Corr")
    for k in range(1,25):
        cols.append(f"{k}0sec Sign %")
    out_df = pd.DataFrame(arr, columns=cols)
    out_df.to_csv(f"tradestation_data/TSLA_{hour}_eval.csv")
    corrs = arr[:, 1:25]
    corr_threshold, corr_sec_dif = np.unravel_index(np.nanargmax(corrs), corrs.shape)
    signs = arr[:, 25:49]
    sign_threshold, sign_sec_dif = np.unravel_index(np.nanargmax(signs), signs.shape)
    return np.nanmax(corrs), price_dif_thresholds[corr_threshold], corr_sec_dif, np.nanmax(signs), price_dif_thresholds[sign_threshold], sign_sec_dif

symbol = "TSLA"
#price_dif_threshold = .03
price_dif_thresholds = (-np.arange(300) / 1000) + .2
size_threshold = 1
# for size_threshold in range(1, 100, 1):
hour = 13
days = ["13", "14", "15", "16"]
dates = [f"2023-06-{day}" for day in days]
print(f"{hour}: {eval_signal(symbol, dates, hour, price_dif_thresholds, size_threshold)}")
days = ["08", "09", "12", "13", "14", "15", "16"]
dates = [f"2023-06-{day}" for day in days]
for hour in range(14, 20):
    print(f"{hour}: {eval_signal(symbol, dates, hour, price_dif_thresholds, size_threshold)}")


13: (0.6260966760333093, 0.16, 0, 0.9411764705882353, 0.15500000000000003, 1)
14: (0.999136253028451, 0.2, 5, 1.0, 0.2, 0)
15: (1.0000000000000002, 0.17900000000000002, 19, 1.0, 0.185, 0)
16: (0.9999999201564999, 0.2, 7, 1.0, 0.2, 8)
17: (1.0000000000000002, 0.129, 0, 1.0, 0.14300000000000002, 0)
18: (1.0, 0.185, 2, 1.0, 0.2, 2)
19: (1.0, 0.16, 2, 1.0, 0.199, 0)


In [48]:
import numpy as np
import pandas as pd
def calc_gain_long(df, buy_threshold, sell_threshold, size_threshold):
    last_sell = df.iloc[len(df) - 1, df.columns.get_loc("SELL")]
    df = df.loc[((df["Side"] == "Bid") & (df["Dif"] >= buy_threshold))
                | ((df["Side"] == "Ask") & (df["Dif"] <= sell_threshold))]
    df.loc[df["Side"] == "Bid", "Multiplier"] = df.loc[df["Side"] == "Bid", "Dif"] - buy_threshold
    df.loc[df["Side"] == "Ask", "Multiplier"] = df.loc[df["Side"] == "Ask", "Dif"] - sell_threshold
    df["VolumeSum"] = df["TotalSize"] * df["Multiplier"]
    df = df[["Time", "VolumeSum", "BUY", "SELL"]]
    df = df.groupby("Time").agg({"VolumeSum": "sum", "BUY": "first", "SELL": "first"}).reset_index()
    df = df[(df["VolumeSum"] > size_threshold) | (df["VolumeSum"] < -size_threshold)]
    df["Long"] = df["VolumeSum"] > size_threshold
    df["Short"] = df["VolumeSum"] < -size_threshold
    if len(df) < 2:
        return 0, None

    df["Buy Action"] = df["Long"] & df["Short"].shift()
    if df.iloc[0, df.columns.get_loc("Long")]:
        df.iloc[0, df.columns.get_loc("Buy Action")] = True
    df["Sell Action"] = df["Short"] & df["Long"].shift()
    df.loc[df["Buy Action"], "Gain"] = -pd.to_numeric(df.loc[df["Buy Action"], "BUY"]) 
    df.loc[df["Sell Action"], "Gain"] = pd.to_numeric(df.loc[df["Sell Action"], "SELL"])

    if df.iloc[len(df) - 1, df.columns.get_loc("Long")]:
        df.loc[len(df)] = [0, 0, 0, 0, 0, 0, 0, 0, last_sell]
    return df["Gain"].sum(), df

def calc_gain_short(df, buy_to_cover_threshold, sell_short_threshold, size_threshold):
    last_buy_to_cover = df.iloc[len(df) - 1, df.columns.get_loc("BUY")]
    df = df.loc[((df["Side"] == "Bid") & (df["Dif"] >= buy_to_cover_threshold))
                | ((df["Side"] == "Ask") & (df["Dif"] <= sell_short_threshold)) ]
    df.loc[df["Side"] == "Bid", "Multiplier"] = df.loc[df["Side"] == "Bid", "Dif"] - buy_to_cover_threshold
    df.loc[df["Side"] == "Ask", "Multiplier"] = df.loc[df["Side"] == "Ask", "Dif"] - sell_short_threshold
    df["VolumeSum"] = df["TotalSize"] * df["Multiplier"]
    df = df[["Time", "VolumeSum", "BUY", "SELL"]]
    df = df.groupby("Time").agg({"VolumeSum": "sum", "BUY": "first", "SELL": "first"}).reset_index()
    df = df[(df["VolumeSum"] > size_threshold) | (df["VolumeSum"] < -size_threshold)]
    df["Long"] = df["VolumeSum"] > size_threshold
    df["Short"] = df["VolumeSum"] < -size_threshold
    if len(df) < 2:
        return 0, None

    df["Sell Action"] = df["Short"] & df["Long"].shift()
    if df.iloc[0, df.columns.get_loc("Short")]:
        df.iloc[0, df.columns.get_loc("Sell Action")] = True
    df["Buy Action"] = df["Long"] & df["Short"].shift()
    df.loc[df["Buy Action"], "Gain"] = -pd.to_numeric(df.loc[df["Buy Action"], "BUY"]) 
    df.loc[df["Sell Action"], "Gain"] = pd.to_numeric(df.loc[df["Sell Action"], "SELL"])

    if df.iloc[len(df) - 1, df.columns.get_loc("Short")]:
        df.loc[len(df)] = [0, 0, 0, 0, 0, 0, 0, 0, -last_buy_to_cover]
    return df["Gain"].sum(), df

In [30]:
import numpy as np
import pandas as pd
def find_thresholds(symbol, dates, hour, buy_thresholds, sell_thresholds, size_threshold):
    collection_1 = f"{symbol}_10sec_ts_lvl1"
    collection_2 = f"{symbol}_10sec_ts_lvl2"
    collection_3 = f"{symbol}_ts_trade_prices"
    df_dict = {}
    for date in dates:
        start = f'{date}T{hour}:00:00Z'
        end = f'{date}T{hour+1}:00:00Z'
        query_1 = {'$and': [{'CurTime': {'$gte': start}}, {'CurTime': {'$lt': end}}]}
        df_1 = pd.DataFrame(list(mongo.mongoDB[collection_1].find(query_1)))
        df_2 = pd.DataFrame(list(mongo.mongoDB[collection_2].find(query_1)))
        df = pd.merge(df_1, df_2, on="CurTime")
        df["Dif"] = (df["Price"] - df["Last"])
        df = df.rename(columns={"CurTime": "Time"})
        query_3 = {'$and': [{'Time': {'$gte': start}}, {'Time': {'$lt': end}}]}
        df_3 = pd.DataFrame(list(mongo.mongoDB[collection_3].find(query_3)))
        df = pd.merge(df, df_3, on="Time")
        df = df[["Time", "Side", "TotalSize", "Dif", "BUY", "SELL"]]
        df = df[df["BUY"].notna() & df["SELL"].notna()]
        df_dict[date] = df

    long_arr = np.zeros((len(buy_thresholds) * len(sell_thresholds), len(dates) + 3))
    long_arr[:, 0] = np.repeat(buy_thresholds, len(sell_thresholds))
    long_arr[:, 1] = np.tile(sell_thresholds, len(buy_thresholds))
    short_arr = np.zeros((len(buy_thresholds) * len(sell_thresholds), len(dates) + 3))
    short_arr[:, 0] = np.repeat(buy_thresholds, len(sell_thresholds))
    short_arr[:, 1] = np.tile(sell_thresholds, len(buy_thresholds))
    for i in range(len(long_arr)):
        buy_threshold, sell_threshold = long_arr[i, 0], long_arr[i, 1]
        for j, date in enumerate(dates):
            long_arr[i, j + 2], _ = calc_gain_long(df_dict[date].copy(), buy_threshold, sell_threshold, size_threshold)
            short_arr[i, j + 2], _ = calc_gain_short(df_dict[date].copy(), buy_threshold, sell_threshold, size_threshold)
    long_arr[:, -1] = long_arr[:, 2:-1].sum(axis=1)
    long_df = pd.DataFrame(long_arr, columns = ["Buy Threshold", "Sell Threshold"]+dates+["Sum"])
    long_df = long_df.sort_values(by=["Sum"], ascending=False)
    long_df.to_csv(f"tradestation_data/TSLA_{hour}_thresholds_long.csv")
    short_arr[:, -1] = short_arr[:, 2:-1].sum(axis=1)
    short_df = pd.DataFrame(short_arr, columns = ["Buy Threshold", "Sell Threshold"]+dates+["Sum"])
    short_df = short_df.sort_values(by=["Sum"], ascending=False)
    short_df.to_csv(f"tradestation_data/TSLA_{hour}_thresholds_short.csv")
    return long_df.iloc[0].tolist(), short_df.iloc[0].tolist()

symbol = "TSLA"
buy_thresholds = (-np.arange(50) / 100) + .25
sell_thresholds = (-np.arange(50) / 100) + .25
size_threshold = 0
#for size_threshold in range(1, 100, 1):
hour = 13
days = ["13", "14", "15", "16", "20", "21", "22"]
dates = [f"2023-06-{day}" for day in days]
print(f"{hour}: {find_thresholds(symbol, dates, hour, buy_thresholds, sell_thresholds, size_threshold)}")
days = ["08", "09", "12", "13", "14", "15", "16", "20", "21", "22"]
dates = [f"2023-06-{day}" for day in days]
for hour in range(14, 20):
    print(f"{hour}: {find_thresholds(symbol, dates, hour, buy_thresholds, sell_thresholds, size_threshold)}")


13: ([0.04000000000000001, -0.06, 0.25999999999993406, -0.9500000000000455, 6.440000000000111, 0.7799999999999159, 3.429999999999893, -1.8500000000002501, 3.9499999999999886, 12.059999999999548], [-0.24, -0.09999999999999998, 0.0, 0.0, -0.22999999999998977, 0.040000000000020464, 0.009999999999990905, 1.8199999999999932, 0.36000000000001364, 2.0000000000000284])
14: ([0.12, -0.08000000000000002, 0.0, 3.219999999999999, -1.2800000000000296, 0.2400000000000091, 0.3199999999999932, 0.5, 0.7599999999999909, 0.0, 1.5, 0.0, 5.2599999999999625], [0.23, -0.12, 0.0, 0.0, 2.3000000000000114, 0.6999999999999886, 0.0, 0.8000000000000114, 2.6999999999999886, 0.0, 1.169999999999959, 2.680000000000007, 10.349999999999966])
15: ([0.01999999999999999, -0.10999999999999999, 1.2199999999999989, -0.7699999999999818, 0.24000000000003752, 1.5699999999999648, -1.5900000000000318, 0.7899999999999636, 0.20000000000004547, 1.3500000000000227, -0.5999999999999659, 5.919999999999959, 8.330000000000013], [0.19, -0.

In [49]:
import numpy as np
import pandas as pd
def test_thresholds(symbol, dates, hour, buy_threshold, sell_threshold, buy_to_cover_threshold, sell_short_threshold, size_threshold):
    collection_1 = f"{symbol}_10sec_ts_lvl1"
    collection_2 = f"{symbol}_10sec_ts_lvl2"
    collection_3 = f"{symbol}_ts_trade_prices"
    long_arr = np.empty((0,7))
    short_arr = np.empty((0,7))
    for date in dates:
        start = f'{date}T{hour}:00:00Z'
        end = f'{date}T{hour+1}:00:00Z'
        query_1 = {'$and': [{'CurTime': {'$gte': start}}, {'CurTime': {'$lt': end}}]}
        df_1 = pd.DataFrame(list(mongo.mongoDB[collection_1].find(query_1)))
        df_2 = pd.DataFrame(list(mongo.mongoDB[collection_2].find(query_1)))
        df = pd.merge(df_1, df_2, on="CurTime")
        df["Dif"] = (df["Price"] - df["Last"])
        df = df.rename(columns={"CurTime": "Time"})
        query_3 = {'$and': [{'Time': {'$gte': start}}, {'Time': {'$lt': end}}]}
        df_3 = pd.DataFrame(list(mongo.mongoDB[collection_3].find(query_3)))
        df = pd.merge(df, df_3, on="Time")
        df = df[["Time", "Side", "TotalSize", "Dif", "BUY", "SELL"]]
        df = df[df["BUY"].notna() & df["SELL"].notna()]

        _, long_df = calc_gain_long(df.copy(), buy_threshold, sell_threshold, size_threshold)
        if long_df is not None:
            long_df = long_df[["Time", "VolumeSum", "BUY", "SELL", "Buy Action", "Sell Action", "Gain"]]
            long_arr = np.concatenate((long_arr, long_df.to_numpy()), axis=0)
        _, short_df = calc_gain_short(df.copy(), buy_to_cover_threshold, sell_short_threshold, size_threshold)
        if short_df is not None:
            short_df = short_df[["Time", "VolumeSum", "BUY", "SELL", "Buy Action", "Sell Action", "Gain"]]
            short_arr = np.concatenate((short_arr, short_df.to_numpy()), axis=0)
    long_df = pd.DataFrame(long_arr, columns=["Time", "VolumeSum", "BUY", "SELL", "Buy Action", "Sell Action", "Gain"])
    long_df.to_csv(f"tradestation_data/TSLA_{hour}_test_long.csv")
    short_df = pd.DataFrame(short_arr, columns=["Time", "VolumeSum", "BUY", "SELL", "Buy Action", "Sell Action", "Gain"])
    short_df.to_csv(f"tradestation_data/TSLA_{hour}_test_short.csv")
symbol = "TSLA"
buy_thresholds = {13: .04, 14: .12, 15: .02, 16: .04, 17: .03, 18: .03, 19: .02}
sell_thresholds = {13: -.06, 14: -.08, 15: -.11, 16: -.07, 17: -.09, 18: -.22, 19: -.05}
buy_to_cover_thresholds = {13: -.24, 14: .23, 15: .19, 16: .22, 17: .03, 18: .08, 19: .02}
sell_short_thresholds = {13: -.1, 14: -.12, 15: -.12, 16: -.08, 17: -.09, 18: -.18, 19: -.05}
size_threshold = 0
hour = 13
days = ["13", "14", "15", "16", "20", "21", "22"]
dates = [f"2023-06-{day}" for day in days]
test_thresholds(symbol, dates, hour, buy_thresholds[hour], sell_thresholds[hour], buy_to_cover_thresholds[hour], sell_short_thresholds[hour], size_threshold)
days = ["08", "09", "12", "13", "14", "15", "16", "20", "21", "22"]
dates = [f"2023-06-{day}" for day in days]
for hour in range(14, 20):
    test_thresholds(symbol, dates, hour, buy_thresholds[hour], sell_thresholds[hour], buy_to_cover_thresholds[hour], sell_short_thresholds[hour], size_threshold)

In [14]:
def get_max_volume(symbol, start_date, end_date):
    collection_1 = f"{symbol}_10sec_ts_lvl1"
    collection_2 = f"{symbol}_10sec_ts_lvl2"
    start = f'2023-06-{start_date}T13:30:00Z'
    end = f'2023-06-{end_date}T20:00:00Z'
    query = {'$and': [{'CurTime': {'$gte': start}}, {'CurTime': {'$lt': end}}]}
    df_1 = pd.DataFrame(list(mongo.mongoDB[collection_1].find(query)))
    df_2 = pd.DataFrame(list(mongo.mongoDB[collection_2].find(query)))
    df = pd.merge(df_1, df_2, on="CurTime")
    df.loc[df["Side"] == "Bid", "Dif"] = (df["Price"] - df["Last"])
    df.loc[df["Side"] == "Ask", "Dif"] = (df["Last"] - df["Price"])
    return df["TotalSize"].max(), df["TotalSize"].min(), df["TotalSize"].mean(), df["Dif"].max(), df["Dif"].min(), df["Dif"].mean() 

symbol = "TSLA"
start_date = "08"
end_date = "14"
print(get_max_volume(symbol, start_date, end_date))

(225600.0, 1.0, 487.7769264200571, 5.219999999999999, -250.45, -3.205123899786772)


In [8]:
def get_spread(symbol, start_date, end_date):
    collection_1 = f"{symbol}_10sec_ts_lvl1"
    collection_2 = f"{symbol}_10sec_ts_lvl2"
    collection_3 = f"{symbol}_ts_trade_prices"
    start = f'2023-06-{start_date}T13:30:00Z'
    end = f'2023-06-{end_date}T20:00:00Z'
    query = {'$and': [{'CurTime': {'$gte': start}}, {'CurTime': {'$lt': end}}]}
    df_1 = pd.DataFrame(list(mongo.mongoDB[collection_1].find(query)))
    df_2 = pd.DataFrame(list(mongo.mongoDB[collection_2].find(query)))
    query = {'$and': [{'Time': {'$gte': start}}, {'Time': {'$lt': end}}]}
    df_3 = pd.DataFrame(list(mongo.mongoDB[collection_3].find(query)))
    df_3 = df_3.rename(columns={"Time": "CurTime"})
    df = pd.merge(df_1, df_2, on="CurTime")
    print(df)
    df = pd.merge(df, df_3, on="CurTime")
    print(df)
    df = df[df["BUY"].notnull() & df["SELL"].notnull() & df["Last"].notnull()]
    df.loc[df["Side"] == "Bid", "Dif"] = (df["Price"] - df["Last"])
    df.loc[df["Side"] == "Ask", "Dif"] = (df["Last"] - df["Price"])
    df = df.loc[df["Dif"] >= 0]
    df["BUY"] = pd.to_numeric(df["BUY"])
    df["SELL"] = pd.to_numeric(df["SELL"])
    df["Buy Dif"] = df["BUY"] - df["Last"]
    df["Sell Dif"] = df["Last"] - df["SELL"]
    df["Spread"] = df["BUY"] - df["SELL"]
    print(df["BUY"].min())
    print(df["Last"].min())
    return df["Buy Dif"].max(), df["Buy Dif"].min(), df["Buy Dif"].mean(), df["Sell Dif"].max(), df["Sell Dif"].min(), df["Sell Dif"].mean(), df["Spread"].max(), df["Spread"].min(), df["Spread"].mean()

symbol = "TSLA"
start_date = "13"
end_date = "14"
print(get_spread(symbol, start_date, end_date))

                           _id_x               CurTime             GetTime_x  \
0       64886f587199f685ff39196a  2023-06-13T13:30:00Z  2023-06-13T13:29:59Z   
1       64886f587199f685ff39196a  2023-06-13T13:30:00Z  2023-06-13T13:29:59Z   
2       64886f587199f685ff39196a  2023-06-13T13:30:00Z  2023-06-13T13:29:59Z   
3       64886f587199f685ff39196a  2023-06-13T13:30:00Z  2023-06-13T13:29:59Z   
4       64886f587199f685ff39196a  2023-06-13T13:30:00Z  2023-06-13T13:29:59Z   
...                          ...                   ...                   ...   
151611  648a1c367199f685ff4cb001  2023-06-14T19:59:50Z  2023-06-14T19:59:49Z   
151612  648a1c367199f685ff4cb001  2023-06-14T19:59:50Z  2023-06-14T19:59:49Z   
151613  648a1c367199f685ff4cb001  2023-06-14T19:59:50Z  2023-06-14T19:59:49Z   
151614  648a1c367199f685ff4cb001  2023-06-14T19:59:50Z  2023-06-14T19:59:49Z   
151615  648a1c367199f685ff4cb001  2023-06-14T19:59:50Z  2023-06-14T19:59:49Z   

       Symbol             TradeTime    